In [11]:
#Hybrid Recommender is a combination of two or more recommender systems
#This examples uses content and collaborative recommender systems
#This RS will recommend films based on the voting average and user's past watching history

In [12]:
#Common libraries
import pandas as pd
import numpy as np
from surprise import SVD, Reader, Dataset

In [13]:
cosine_sim = pd.read_csv('cosine_sim.csv')

In [14]:
#Incorporating other datasets
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)
#Later used for the function
svd = SVD()
train_set = data.build_full_trainset()
svd.fit(train_set)

In [15]:
#Reoragnizing the output
id_map = pd.read_csv('movie_ids.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [16]:
smd = pd.read_csv('metadata_small.csv')

In [19]:
#the function for the hybrid recommender system
def hybrid(user_id, title):
    idx = cosine_sim_map[title]    
    sim_scores = list(enumerate(cosine_sim[str(int(idx))]))    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)    
    sim_scores = sim_scores[1:26]    
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]    
    movies['est'] = movies['id'].apply(lambda x: svd.predict(user_id, id_to_title.loc[x]['movieId']).est)    
    movies = movies.sort_values('est', ascending=False)    
    return movies.head(10)

In [20]:
#The first parameter is the user_id and the second is the title of the film
#
hybrid(2, 'Inception')

,title,vote_count,vote_average,year,id,est
6640,Déjà Vu,1519.0,6.6,2006,7551,3.637194
5924,Fortress,171.0,5.7,1992,12088,3.604558
4173,Minority Report,2663.0,7.1,2002,180,3.562025
5580,The Three Lives of Thomasina,12.0,6.8,1963,15081,3.561796
7502,The Book of Eli,2207.0,6.6,2010,20504,3.535351
8207,Looper,4777.0,6.6,2012,59967,3.520219
7828,I Am Number Four,1606.0,5.9,2011,46529,3.498894
8500,Don Jon,1708.0,5.9,2013,138697,3.487519
6967,Doomsday,374.0,5.8,2008,13460,3.461785
7901,Super 8,2496.0,6.6,2011,37686,3.455279
